<a href="https://colab.research.google.com/github/andreagrusso/GPT2_word_saliency_fMRI/blob/main/90s_surp_saliency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 25.7 MB/s 
     |████████████████████████████████| 3.3 MB 20.6 MB/s 
     |████████████████████████████████| 56 kB 3.5 MB/s 
     |████████████████████████████████| 596 kB 27.4 MB/s 
     |████████████████████████████████| 895 kB 26.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from torch.nn import functional as F
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead,GPT2LMHeadModel
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import pickle, glob, json, csv

Import the GPT2 model from https://huggingface.co/LorenzoDeMattei/GePpeTto Paper: https://arxiv.org/abs/2004.14253 "GePpeTto Carves Italian into a Language Model"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('LorenzoDeMattei/GePpeTto')
model = GPT2LMHeadModel.from_pretrained("LorenzoDeMattei/GePpeTto")#.to('cuda')

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/534k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/280k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/463M [00:00<?, ?B/s]

Import the text of the narrative "gianna_orig.txt". This file contain the full text of the narrative including punctuation marks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
f=open('/content/drive/My Drive/Colab Notebooks/GPT/gianna_noVirgoletteParentesi_v02.txt', 'r',encoding='utf-8-sig')
orig_text = f.read() # the text is read as whole

In [ ]:
full_words = orig_text.split(' ')

#print(full_words)

# initialize a null list
unique_words = []
  
# traverse for all elements
for x in full_words:
    # check if exists in unique_list or not
    if x not in unique_words:
        unique_words.append(x.lower())

print(unique_words)

['in', 'principio,', 'avevo', 'solo', 'due', 'chiavi', 'quella', 'della', 'porta', 'di', 'casa', 'e', 'del', 'portone', 'palazzo.', 'ma', 'non', 'lavoro.', 'facevo', 'un', 'part', 'time', 'nero', 'negozio', 'giocattoli,', 'figuriamoci', 'se', 'potevo', 'pretendere', 'che', 'mi', 'dessero', 'le', 'chiavi.', 'e', 'poi,', 'scusate', 'che?', 'era', 'uno', 'quei', 'grandi', 'magazzini', 'sulla', 'variante,', 'sembrano', 'sempre', 'aperti,', 'o', 'almeno', 'li', 'vedete', 'mai', 'chiusi,', 'con', 'porte', 'automatiche', 'tutto.', 'che', 'bisogno', "c'era", 'una', 'chiave?', 'poi,', 'sapete,', 'è', 'lavoro,', 'realtà', 'nemmeno', 'full', 'nero,', 'lo', 'si', 'dice', 'per', 'consolarsi', 'vado', 'al', 'poi', 'tutto', 'crolla', 'quando', 'qualcuno', 'diciamo', 'mia', 'mamma', 'voce', 'candida', "po'", 'sognante', 'quando', 'inizierai', 'a', 'lavorare...', 'come', 'volessi', 'lavorare.\nquindi,', 'delle', 'prime', 'cose,', 'anzi', 'forse', 'proprio', 'la', 'prima', 'il', 'dottor', 'vigliani', 'h

**Please note tha Goldestein et al. (2021) proved that the correlation between the predictions of the GPT2 and the human predictions increases as the context window increases by reaching an asymptote of r = .79 at 100-word context **
We could use a window of that size

We will try to use a fixed context of time (e.g. 30 s) instead of a fixed context of tokens

In [ ]:
all_tokens_timestamps = pd.read_csv('/content/drive/My Drive/Colab Notebooks/GPT/all_tokens+timestamp.txt',delimiter='\t')
timestamps=all_tokens_timestamps['time']
duration = all_tokens_timestamps['duration']

In [ ]:
input_ids = tokenizer.encode(orig_text, return_tensors="pt")#.to('cuda')


In [ ]:

time_window = 90 #seconds
#we normalize respect the first word time. Thus the first word will have a timestamps equals to zero 
timestamps_zerobased = timestamps-timestamps[0]

#we need to find the index of the word at the begining of the time window
#we should first consider all the words from 0 to 15 seconds, then all the word
#after 30 seconds


#get the word at 30 seconds
first_word_id = np.abs(timestamps_zerobased-time_window).argmin()

#define variable for next word probabilities
next_word_probs = []

#model the token contained in the 30s time window (first window)
all_next_token_logits = model(input_ids[:,:first_word_id+1])[0].requires_grad_(True)

#loop until the penultimate word contained in the window to get sequential next word probs
for c_idx,c_word in enumerate(input_ids[0][:first_word_id+1]):
  
  word2pred = all_tokens_timestamps.iloc[c_idx]['token']
  context_words = all_tokens_timestamps.iloc[:c_idx]['token'].tolist()

  #softmax on the probabilities assigned to all words of the vocabulary given the context
  next_token_probs = F.softmax(all_next_token_logits[:,c_idx,:],dim=-1)
  next_word_probs.append([next_token_probs[:,input_ids[0][c_idx+1]].item(),
                         word2pred,
                         timestamps[first_word_id],
                         timestamps[c_idx],
                         duration[c_idx], 
                         context_words])
  




#when the first loop is finished we have analyzed the next-word probabilities
#of all the words contained in the window so we need to move the time-window


#loop on the remaining words

for i in range(first_word_id+1,input_ids.size()[1]-1): #skip the last period character
  
  curr_diff = timestamps_zerobased[i]-time_window
  start_idx = np.abs(timestamps_zerobased-curr_diff).argmin()

  word2pred = all_tokens_timestamps.iloc[i]['token']
  context_words = all_tokens_timestamps.iloc[start_idx:i]['token'].tolist()


  #consider only the tokens that fall in time window of 30 seconds
  all_next_token_logits = model(input_ids[:,start_idx:i])[0].requires_grad_(True)
  #perform a softmax on the logits assigned to each word of the vocabulary
  next_token_probs = F.softmax(all_next_token_logits[:,-1,:],dim=-1)
  #get the probability of the next word, the next word and the context words 
  next_word_probs.append([next_token_probs[:,input_ids[0][i+1]].item(),
                         word2pred,
                        timestamps[start_idx],
                        timestamps[i],
                        duration[i],
                        context_words])

In [ ]:
word_prob = pd.DataFrame({'token_predicted' : [tmp[1] for tmp in next_word_probs],
                          'probability':np.array([tmp[0] for tmp in next_word_probs]),
                          'duration':np.array([tmp[4] for tmp in next_word_probs]),
                          'context_start_time':np.array([tmp[2] for tmp in next_word_probs]),
                          'context_stop_time':np.array([tmp[3] for tmp in next_word_probs])})

f = open('/content/drive/My Drive/Colab Notebooks/GPT/raw_probabilities_context_words_win'+str(time_window)+'s.txt','w')

context_words = []

for tmp in next_word_probs:
    context_words.append(tmp[-1])
    for w in tmp[-1]:
      f.write(w +'\t')
    
    f.write('\n')
f.close()

word_prob.to_csv('/content/drive/My Drive/Colab Notebooks/GPT/raw_probabilities_win'+str(time_window)+'s.csv')

In [ ]:
word_prob.head(20)

,token_predicted,probability,duration,context_start_time,context_stop_time
0,In,0.000334,0.15,101.8,11.80
1,principio,0.064819,0.66,101.8,11.95
2,",",0.000491,0.09,101.8,12.61
3,avevo,0.013082,0.33,101.8,12.70
4,solo,0.032139,0.27,101.8,13.03
5,due,0.000318,0.18,101.8,13.30
6,chiavi,0.000005,0.45,101.8,13.48
7,quella,0.053214,0.30,101.8,14.35
8,della,0.042729,0.21,101.8,14.65
9,porta,0.058577,0.33,101.8,14.86


Multiply the probabilities associated to the tokens in which a single word has been divided. E.g. "ga"++"tto"

In [ ]:
import csv
tok2remove = []
with open('/content/drive/My Drive/Colab Notebooks/GPT/token2remove.txt') as csvfile:
   csvreader = csv.reader(csvfile, delimiter=',')
   for row in csvreader:
    tok2remove.append(row)

tok2remove = [int(item) for tmp in tok2remove for item in tmp]

#remove rows with multi tokens words
new_probs = word_prob.drop(tok2remove)

In [ ]:
new_probs.head(20)
new_probs.to_csv('/content/drive/My Drive/Colab Notebooks/GPT/clean_probabilities_win' + str(time_window) + 's.csv')

In [ ]:
clean_context_words = [context_words[i] for i in range(len(context_words)) if i not in tok2remove]

f = open('/content/drive/My Drive/Colab Notebooks/GPT/clean_probabilities_context_words_win' + str(time_window) + 's.txt','w')

for tmp in clean_context_words:
  
    for w in tmp:
      f.write(w +'\t')
    
    f.write('\n')
f.close()

Now we need to extract the saliency for each token of the test. The output would be likely a dictionary where the key is the token and the value is the salience vector for that token and the input sequence of tokens

In [ ]:
time_window = 90 #seconds

#we normalize respect the first word time. Thus the first word will have a timestamps equals to zero 
timestamps_zerobased = timestamps-timestamps[0]

#we need to find the index of the word at the begining of the time window
#we should first consider all the words from 0 to 30 seconds, then all the word
#after 30 seconds

#get the word at 30 seconds
first_word_id = np.abs(timestamps_zerobased-time_window).argmin()

#back to cpu
model = GPT2LMHeadModel.from_pretrained("LorenzoDeMattei/GePpeTto")#.to('cuda')
#get the embedding matrix
embedding_matrix = model.transformer.wte.weight
#then generate a one hot tensor to select only the input token for the considered token
vocab_size = embedding_matrix.shape[0]

#get the all the input ids
input_ids = tokenizer.encode(orig_text, return_tensors="pt")#.to('cuda')

#loop here with the sliding window

In [ ]:
#one hot tensor
one_hot_tensor = F.one_hot(input_ids[:,:first_word_id], num_classes=vocab_size).to(dtype=torch.float32).requires_grad_(True)

#embedding of the inputs
inputs_embeds = torch.matmul(one_hot_tensor, embedding_matrix).requires_grad_(True)
inputs_embeds.retain_grad()


token_saliency = []
#loop until the penultimate word contained in the window to get sequential next word probs
for c_idx,c_word in enumerate(input_ids[0][:first_word_id+1]):
  
  next_input_ids = input_ids[:,c_idx]
  
  word2pred = all_tokens_timestamps.iloc[c_idx]['token']
  context_words = all_tokens_timestamps.iloc[:c_idx]['token'].tolist()

  grad = torch.autograd.grad(model(inputs_embeds=inputs_embeds)[0][:,c_idx-1,next_input_ids],inputs_embeds)[0]
  grad_x_input = grad*inputs_embeds
  importance_raw = torch.norm(grad_x_input,dim=2)
  importance_norm = importance_raw/torch.sum(importance_raw)
  token_saliency.append([word2pred,
                           importance_norm,
                         0,
                         timestamps[c_idx],
                           context_words,
                         np.arange(0,c_idx)])

#dump the first 30 seconds
pickle.dump(token_saliency,open('/content/drive/My Drive/Colab Notebooks/GPT/gradients_first0.' + str(time_window) + 's.pkl','wb'))


In [ ]:
a=list(range(2200,input_ids.size()[1]-first_word_id-1,1))
print(input_ids.size()[1])
print(first_word_id-1)
a.append(input_ids.size()[1])
print(a)

2512
307
[2200, 2201, 2202, 2512]


In [ ]:
#the real sliding window procedure after the first window
#loop on the remaining words

#to avoid a memory issue we define a set of index at which we dump the results and clean the output variable
parts = list(range(2200,input_ids.size()[1],50))#list(range(100,input_ids.size()[1]-first_word_id-1,100))
print(parts)
parts.append(input_ids.size()[1])

#define output variable
token_saliency = []

for i in range(2201,input_ids.size()[1]-1): #range(first_word_id+1,input_ids.size()[1]-1):
  
  #get the initial time of the context
  curr_diff = timestamps_zerobased[i]-time_window
  start_idx = np.abs(timestamps_zerobased-curr_diff).argmin()

  word2pred = all_tokens_timestamps.iloc[i]['token'] #we want to predict the i-th word, thus the context is up to the i-th with the i-th excluded
  context_words = all_tokens_timestamps.iloc[start_idx:i]['token'].tolist()

  #id of the word to predict
  next_input_ids = input_ids[:,i]

  #one hot tensor for the tokens contained in the window
  one_hot_tensor = F.one_hot(input_ids[:,start_idx:i], num_classes=vocab_size).to(dtype=torch.float32).requires_grad_(True)


  #embedding of the inputs
  inputs_embeds = torch.matmul(one_hot_tensor, embedding_matrix).requires_grad_(True)
  inputs_embeds.retain_grad()

  grad = torch.autograd.grad(model(inputs_embeds=inputs_embeds)[0][:,-1,next_input_ids],
                             inputs_embeds)[0]
  grad_x_input = grad*inputs_embeds
  importance_raw = torch.norm(grad_x_input,dim=2)
  importance_norm = importance_raw/torch.sum(importance_raw)
  token_saliency.append([word2pred,
                           importance_norm,
                           timestamps[start_idx],
                          timestamps[i],
                           context_words,
                         np.arange(start_idx,i)])                                                     

  if i in parts:
    print('dump',i)
    print('length',len(token_saliency))
    pickle.dump(token_saliency,open('/content/drive/My Drive/Colab Notebooks/GPT/gradients_upto'+str(timestamps[i])+'words.pkl','wb'))
    token_saliency = []

#last token    
pickle.dump(token_saliency,open('/content/drive/My Drive/Colab Notebooks/GPT/gradients_upto'+str(timestamps[i])+'words.pkl','wb'))



#estimate the gradient*input that gives the importance of each input token for the selected output 
#https://jalammar.github.io/explaining-transformers/ https://github.com/jalammar/ecco/blob/main/src/ecco/attribution.py 
#A Diagnostic Study of Explainability Techniques for Text Classification;
#The elephant in the interpretability room: Why use attention as explanation when we have saliency methods?


[2200, 2250, 2300, 2350, 2400, 2450, 2500]
dump 2250
length 50
dump 2300
length 50
dump 2350
length 50
dump 2400
length 50
dump 2450
length 50
dump 2500
length 50


In [ ]:
saliency_files = glob.glob('/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow' + str(time_window) + '/*gradients*.pkl')
saliency_file = saliency_files.sort()
print(saliency_files)

['/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow90/gradients_first0.90s.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow90/gradients_upto112.72words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow90/gradients_upto128.58words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow90/gradients_upto141.36words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow90/gradients_upto154.92words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow90/gradients_upto169.14words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow90/gradients_upto180.21words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow90/gradients_upto196.38words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow90/gradients_upto208.62words.pkl', '/content/drive/My Drive/Colab Notebooks/GPT/NoQuotMarks_timewindow90/gradients_upto22

In [ ]:
#load and store saliency vector

pred_words = []
raw_saliency_matrix = []
start_times = []
stop_times = []
past_tokens = []
past_tokens_indices = []


for tmp_file in saliency_files:
    tmp_list = pickle.load(open(tmp_file,'rb'))

    for val in tmp_list:
      pred_words.append(val[0])
      raw_saliency_matrix.append(val[1].detach().numpy())
      start_times.append(val[2])
      stop_times.append(val[3])
      past_tokens.append(val[4])
      past_tokens_indices.append(val[5])

In [ ]:
#save the previous outputs
f = open('/content/drive/My Drive/Colab Notebooks/GPT/raw_salience_win' + str(time_window) + 's.txt',"w")

for i,vec in enumerate(raw_saliency_matrix):
  f.write(pred_words[i] + ',' + str(start_times[i]) + ',' + str(stop_times[i]) + ',')
  for el in vec:
    f.write(str(el))
  f.write('\n')

f.close()

f = open('/content/drive/My Drive/Colab Notebooks/GPT/raw_past_tokens_win' + str(time_window) + 's.txt',"w")

for i,tmp_list in enumerate(past_tokens):
  f.write(pred_words[i]+'\t')
  for val in tmp_list:
    f.write(val + '\t')
  f.write('\n')

f.close()


#save also as pickle file
raw_salience_list = [[pred_words[i],start_times[i],stop_times[i],vec,past_tokens_indices[i]]
                     for i,vec in enumerate(raw_saliency_matrix)]
raw_past_token_list =[[pred_words[i],tmp_list]
                      for i,tmp_list in enumerate(past_tokens)]

pickle.dump(raw_salience_list,open('/content/drive/My Drive/Colab Notebooks/GPT/time_window/'+ str(time_window) +'s/raw_salience_win' + str(time_window) + 's.pkl',"wb")) 
pickle.dump(raw_past_token_list,open('/content/drive/My Drive/Colab Notebooks/GPT/time_window/'+ str(time_window) +'s/raw_past_token_win' + str(time_window) + 's.pkl',"wb"))                      

Now we need to clean the clean the saliency matrix to select only the single token word

In [ ]:
#load the indices of the elements to remove
import csv
tok2remove = []
with open('/content/drive/My Drive/Colab Notebooks/GPT/token2remove.txt') as csvfile:
   csvreader = csv.reader(csvfile, delimiter=',')
   for row in csvreader:
    tok2remove.append(row)
tok2remove = [int(item) for tmp in tok2remove for item in tmp]


clean_saliency_matrix = [raw_saliency_matrix[i] for i in range(len(raw_saliency_matrix)) if i not in tok2remove]
clean_past_tokens = [past_tokens[i] for i in range(len(past_tokens)) if i not in tok2remove]
clean_starts = [start_times[i] for i in range(len(start_times)) if i not in tok2remove]
clean_stops = [stop_times[i] for i in range(len(stop_times)) if i not in tok2remove]
clean_pred_words = [pred_words[i] for i in range(len(pred_words)) if i not in tok2remove]
clean_past_token_indices = [past_tokens_indices[i] for i in range(len(past_tokens_indices)) if i not in tok2remove]

#save the clean output
f = open('/content/drive/My Drive/Colab Notebooks/GPT/clean_salience_win' + str(time_window) + 's.txt',"w")

for i,vec in enumerate(clean_saliency_matrix):
  f.write(clean_pred_words[i] + '\t' + str(clean_starts[i]) + '\t' + str(clean_stops[i]) + '\t' + str(vec))
  f.write('\n')

f.close()

f = open('/content/drive/My Drive/Colab Notebooks/GPT/clean_past_tokens_win' + str(time_window) + 's.txt',"w")

for i,tmp_list in enumerate(clean_past_tokens):
  f.write(clean_pred_words[i]+'\t')
  for val in tmp_list:
    f.write(val + '\t')
  f.write('\n')

f.close()

#save also as pickle file
clean_salience_list = [[clean_pred_words[i],clean_starts[i],clean_stops[i],vec,clean_past_token_indices[i]]
                     for i,vec in enumerate(clean_saliency_matrix)]
clean_past_token_list =[[clean_pred_words[i],tmp_list]
                      for i,tmp_list in enumerate(clean_past_tokens)]

pickle.dump(clean_salience_list,open('/content/drive/My Drive/Colab Notebooks/GPT/time_window/' + str(time_window) + 's/clean_salience_win' + str(time_window) + 's.pkl',"wb")) 
pickle.dump(clean_past_token_list,open('/content/drive/My Drive/Colab Notebooks/GPT/time_window/' + str(time_window) + 's/clean_past_token_win' + str(time_window) + 's.pkl',"wb")) 